### Kaggle Data Submission

In [1]:
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt
import seaborn as sns
import pandas.testing as tm
from datetime import datetime as dt
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import plot_importance
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
sns.set()
%matplotlib inline 

C:\Users\sit_k\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
raw_df = pd.read_csv('data/train_df.csv', parse_dates=['date'])

### Import Test Data

In [3]:
test = pd.read_csv('data/test.csv')
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [4]:
raw_df.tail()

,date,date_block_num,shop_id,shop_name,item_id,item_name,item_category_id,item_category_name,item_price,item_cnt_day,sales
2916099,2015-10-22,33,55,Цифровой склад 1С-Онлайн,13093,Карта оплаты Windows: 250 рублей [Цифровая вер...,36,Карты оплаты - Windows (Цифра),250.0,1.0,250.0
2916100,2015-09-21,32,55,Цифровой склад 1С-Онлайн,13091,Карта оплаты Windows: 1000 рублей [Цифровая ве...,36,Карты оплаты - Windows (Цифра),1000.0,1.0,1000.0
2916101,2015-09-16,32,55,Цифровой склад 1С-Онлайн,13094,Карта оплаты Windows: 2500 рублей [Цифровая ве...,36,Карты оплаты - Windows (Цифра),2500.0,1.0,2500.0
2916102,2015-09-22,32,55,Цифровой склад 1С-Онлайн,13094,Карта оплаты Windows: 2500 рублей [Цифровая ве...,36,Карты оплаты - Windows (Цифра),2500.0,2.0,5000.0
2916103,2015-10-26,33,55,Цифровой склад 1С-Онлайн,13092,Карта оплаты Windows: 2000 рублей [Цифровая ве...,36,Карты оплаты - Windows (Цифра),2000.0,1.0,2000.0


### Naive Model

We want to continue to improve on accuracy, but we should set a baseline for naive model in which no algorithm has been run. We will have two naive models: one using Nov 2014 (period 22) and the other using last period (period 33) actuals by product and by shop.

In [5]:
naive = raw_df.query('date_block_num == 22').reset_index()

In [6]:
naive = naive.groupby(['shop_id', 'item_id'])['item_cnt_day'].sum().reset_index()

In [7]:
naive_d = pd.merge(test, naive, on=['shop_id', 'item_id'], how='left')
naive_d.fillna(0,inplace=True)
naive_d.drop(['shop_id', 'item_id'], axis=1, inplace=True)
naive_d.rename(columns={'item_cnt_day':'item_cnt_month'}, inplace=True)
naive_d.to_csv('data/submission_naive.csv', index=False)

In [8]:
naive_LM = raw_df.query('date_block_num == 33').reset_index()
naive_LM = naive_LM.groupby(['shop_id', 'item_id'])['item_cnt_day'].sum().reset_index()
naive_LM = pd.merge(test, naive_LM, on=['shop_id', 'item_id'], how='left')
naive_LM.fillna(0,inplace=True)
naive_LM.drop(['shop_id', 'item_id'], axis=1, inplace=True)
naive_LM.rename(columns={'item_cnt_day':'item_cnt_month'}, inplace=True)
naive_LM.to_csv('data/submission_naiveLM.csv', index=False)

### Time Series Model (SARIMA)

We will use up all the train data and predict Nov 2015 sales. Apply SARIMA model as identified in the model training excerises.

In [9]:
ts_data = raw_df[['date','sales']]
ts_daily = ts_data.groupby(['date'])['sales'].sum()
mod = SARIMAX(ts_daily, order=(3,1,3), seasonal_order=(1,1,2,12), freq='D')
res = mod.fit()
sarimax = res.predict(start='2015-11-01', end='2015-11-30')

C:\Users\sit_k\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\base\model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [10]:
# total sales predicted for Nov 2015
nov_2015 = sarimax.sum(axis=0)

In [11]:
ts_data = raw_df[['date','sales']]
ts_daily = ts_data.groupby(['date'])['sales'].sum()

To split the total sales back to item/shop level, we assume the sales penetration by item by shop is closely resemble to what Oct 2015 was.

In [12]:
oct_2015 = raw_df.query('date_block_num == 33')
oct_2015 = oct_2015.groupby(['item_id', 'shop_id'])['sales'].sum().reset_index()

In [13]:
oct_2015_ttl = oct_2015['sales'].sum()

In [14]:
# get sale % by shop and by item
oct_2015['sales_%'] = oct_2015['sales']/oct_2015_ttl

In [15]:
oct_2015.sort_values(by='sales_%', ascending=False)

,item_id,shop_id,sales,sales_%
10767,7224,25,390738.0,0.005198
1418,1578,42,368959.0,0.004908
10777,7224,42,323928.0,0.004309
1408,1578,25,305966.0,0.004070
10726,7223,25,275931.0,0.003670
...,...,...,...,...
13257,9242,12,-1500.0,-0.000020
24077,16146,21,-1799.0,-0.000024
3360,2860,48,-1799.0,-0.000024
8095,5488,12,-2799.0,-0.000037


In [16]:
sarima_d = pd.merge(test, oct_2015, on=['shop_id','item_id'], how='left')

The following step is to get the price per item_id, which will be used to convers sales to sales count

In [17]:
items = pd.read_csv('./data/items.csv')

In [18]:
items_price = pd.merge(items, raw_df, on=['item_id'], how='left')
items_price = items_price[['item_id', 'item_price']]
items_price = items_price.drop_duplicates(subset='item_id')

In [19]:
sarima_d = pd.merge(sarima_d, items_price, on=['item_id'], how='left')
sarima_d['item_cnt_month'] = nov_2015*sarima_d['sales_%']/sarima_d['item_price']
sarima_d.fillna(0,inplace=True)

In [20]:
sarima_d = sarima_d.reset_index().drop(['shop_id', 'item_id', 'sales', 'sales_%', 'item_price','index'], axis=1)
sarima_d.to_csv('data/submission_sar.csv', index=False)

### Classification Model

In [21]:
final_df = pd.read_csv('data/final_df.csv',index_col=0)
final_df.head(3)

C:\Users\sit_k\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,date_block_num,item_category_id,item_cnt_day,item_price,item_id,shop_id,month,lag1,lag3,lag6,lag12,lag1_pct,lag3_pct,lag6_pct,lag12_pct,target
0,13,40.0,0.0,58.0,0,54,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,14,40.0,0.0,58.0,0,54,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,15,40.0,0.0,58.0,0,54,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### Xgboost

In [22]:
train_final = final_df[final_df.date_block_num <=32]
test_final = final_df[final_df.date_block_num == 33]

In [23]:
train_feature = train_final.drop(columns=['target'])
train_target = train_final['target']
pred_feature = test_final.drop(columns=['target'])
pred_target = test_final['target']

In [24]:
pred_label = test_final.set_index(['item_id','shop_id']).index
DM_train_final = xgb.DMatrix(data=train_feature, label=train_target)
DM_test_final = xgb.DMatrix(data=pred_feature, label=pred_target)

In [25]:
%%time
params = {"booster":"gbtree", "objective":"reg:squarederror", 'max_depth':8, 'n_estimators':500, 'min_child_weight':40, 'colsample_bytree': 0.8, 
          'subsample':0.8, 'eta' :0.3, 'seed':42}

xg_reg = xgb.train(params = params, dtrain=DM_train_final, num_boost_round=15)

Wall time: 17.9 s


In [26]:
preds_final = xg_reg.predict(DM_test_final)

In [27]:
predictions = pd.Series(preds_final).rename('item_cnt_month')
predictions.index = test_final.index
predictions= pd.merge(predictions, test_final[['item_id', 'shop_id']], left_index=True, right_index=True)

In [28]:
test_xgb = pd.merge(test, predictions, on=['item_id', 'shop_id'], how='left')
test_xgb.fillna(0,inplace=True)
test_xgb.drop(columns=['item_id','shop_id'], inplace=True)
test_xgb.head()

,ID,item_cnt_month
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0


In [29]:
test_xgb.to_csv('data/submission_xgb.csv', index=False)

#### Random Forest

In [30]:
rfc = RandomForestRegressor(n_estimators=250, max_depth=6, n_jobs=4, min_samples_leaf=2000)

In [31]:
train_f = train_final.drop(['target'], axis=1)
train_t = train_final['target']

In [39]:
%%time
rfc.fit(train_f, train_t)

Wall time: 25min 41s


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=6, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=2000,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=250, n_jobs=4, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [42]:
pred_rfc = pd.Series(rfc.predict(pred_feature)).rename('item_cnt_month')
pred_rfc.index = test_final.index
pred_rfc = pd.merge(pred_rfc, test_final[['item_id','shop_id']], left_index=True, right_index = True)

In [43]:
test_rfc = pd.merge(test, pred_rfc, on=['item_id', 'shop_id'], how='left')
test_rfc.fillna(0,inplace=True)
test_rfc.drop(columns=['item_id','shop_id'], inplace=True)
test_rfc.head()

,ID,item_cnt_month
0,0,0.251975
1,1,0.000000
2,2,0.926223
3,3,0.097183
4,4,0.000000


In [44]:
test_rfc.to_csv('data/submission_rfc.csv', index=False)

#### Rigde Regression

In [32]:
regr = linear_model.Ridge()
regr.fit(train_f, train_t)
pred_reg = pd.Series(regr.predict(pred_feature)).rename('item_cnt_month')
pred_reg.index = test_final.index
pred_reg = pd.merge(pred_reg, test_final[['item_id','shop_id']], left_index=True, right_index = True)

In [46]:
test_reg = pd.merge(test, pred_reg, on=['item_id', 'shop_id'], how='left')
test_reg.fillna(0,inplace=True)
test_reg.drop(columns=['item_id','shop_id'], inplace=True)
test_reg.head()

,ID,item_cnt_month
0,0,0.306985
1,1,0.000000
2,2,1.031328
3,3,0.029803
4,4,0.000000


In [47]:
test_reg.to_csv('data/submission_reg.csv', index=False)